In [1]:
from __future__ import division, print_function, absolute_import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
ethnea_df = pd.read_csv('names_ethnea_genni_country_sample.csv')
ethnea_df

,AUID,Last,First,Ethnea,Genni,PubCountry
0,12872768_1,_Filho_,_Elias_Abdalla_,HISPANIC,M,Brazil
1,12565879_3,_Bou_Abdallah_,_Jad_,ARAB,M,France
2,17569133_1,_Abdel_Aziz_,_Ayman_,ARAB,M,USA
3,11213157_2,_Abdelmoula_,_Salma_,ARAB,F,Tunisia
4,11439058_1,_Abdou_,_Ibrahim_,ARAB,M,Egypt
5,10919608_3,_Abou_El_Fettouh_,_Hazem_,ARAB,M,USA
6,17314740_3,_Aboutaam_,_Rola_,ARAB,F,France
7,16564217_2,_Alvim_de_Abreu_Silva_Rodrigues_,_Aida_Alexandra_,HISPANIC,F,Brazil
8,6339395_2,_Cristina_Affonso_Scaletsky_,_Isabel_,HISPANIC,F,Brazil
9,7796089_2,_Agbenyega_,_Tsiri_,AFRICAN,-,Ghana


In [10]:
# Start doing DNN for the name embedding
# combine first name and last name
full_name = ethnea_df['First'].str.lower()+ethnea_df['Last'].str.lower()

In [16]:
def extract_structure(word,n_char=4):
    x_struct = []
    word_len = len(word) + n_char
    n_char-=1
    counter = 0
    for i in range(word_len):
        end = i+1
        start = (i - n_char) if (i - n_char) > 0 else 0
        if word[start:end]!='_' and word[start:end]!='':
            x_struct.append(word[start:end])
    return x_struct

#extract_structure('_Elias_Abdalla__Filho_')
full_name_struct = full_name.apply(lambda x: extract_structure(x))

struct_dict = {}
for name_struct_i in full_name_struct:
    for struct_j in name_struct_i:
        if struct_j not in struct_dict:
            struct_dict[struct_j]=0
        struct_dict[struct_j]+=1

In [17]:
len(struct_dict.keys())

48971

In [18]:
struct_dict_keys = list(struct_dict.keys())
ethnic_series = ethnea_df['Ethnea'].str.lower()
ethnic_keys = list(np.unique(ethnic_series.values))

In [19]:
# embedd the structure vocabulary using text embedding and reduce the dimensionality

# convert the names into word structure vector
struct_dict_keys = list(struct_dict.keys())

def transform_structure(name_struct):
    list_structure = []
    for x in name_struct:
        try:
            list_structure.append(struct_dict_keys.index(x)+1)
        except BaseException:
            list_structure.append(0)
    #add pading 0 for structure less than num_input
    #for i in range(len(list_structure),timesteps):
    #    list_structure.append(0)
    return list_structure   
    #return [*map(lambda x:struct_dict_keys.index(x)+1, name_struct)]

#data_source = full_name_struct.apply(lambda x: transform_structure(x))

In [20]:
def transform_labels(x):
    y = np.zeros(len(ethnic_keys))
    y[ethnic_keys.index(x)]=1
    return y

labels = np.array(list(map(lambda x: transform_labels(x),ethnic_series)))

In [9]:
# using tflearn make the graph creation simple
import tensorflow as tf
import tflearn
from tflearn.data_utils import to_categorical, pad_sequences
from sklearn.model_selection import train_test_split

# separate train and training set
#trainX, testX, trainY, testY = train_test_split(data_source,[ethnic_keys.index(x) for x in ethnic_series],test_size = 0.2)
trainIndex, testIndex, trainY, testY = train_test_split(range(len(data_source)),[ethnic_keys.index(x) for x in ethnic_series],test_size = 0.2)



trainX = pad_sequences(data_source[trainIndex], maxlen=50,value=0.)
testX = pad_sequences(data_source[testIndex],maxlen=50,value=0.)
# Converting labels to binary vectors
trainY = to_categorical(trainY,nb_classes=len(ethnic_keys))
testY = to_categorical(testY,nb_classes=len(ethnic_keys))    

NameError: name 'data_source' is not defined

In [16]:
#import pickle
#with open('train_test_fix_index.pickle','wb') as f:
#    pickle.dump((trainIndex,testIndex,trainX,trainY,testX,testY,ethnic_keys,struct_dict_keys),f)

In [7]:
import numpy as np
import keras
from keras.models import Sequential
from keras.models import model_from_json
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

import pickle

with open('train_test_fix_index.pickle', 'rb') as f:
    trainIndex,testIndex,trainX,trainY,testX,testY,ethnic_keys,struct_dict_keys = pickle.load(f)

#with open('traintest-smote.pickle','rb') as f:
#    train_res,test_res = pickle.load(f)

with open('ethnic_keys.pickle','rb') as f:
    name_struct_keys,ethnic_keys = pickle.load(f)
        
embedding_vector_length = 1000
lstm_layer = 1000
max_sequence = 50

In [4]:
import random
from tflearn.data_utils import to_categorical, pad_sequences
def generateNoise(full_name_source,freq):
    str_len = len(full_name_source)
    str_freq = round(freq * str_len);
    str_noise = random.sample(range(str_len),str_freq)
    name_list = np.array(list(full_name_source))
    #print(name_list)
    name_list[str_noise] = '_'
    return ''.join(name_list)
#full_name[trainIndex]

In [13]:
#?random.sample
generateNoise(full_name[0],0.2)
# convert categorical to binary crossentropy
#trainY = np.array([np.where(x>0)[0][0] for x in trainY])
#testY = np.array([np.where(x>0)[0][0] for x in testY])

#test_res


'_elia__abdalla__fi_h__'

In [5]:
def batch_noise(full_name,noise=0.1):
#struct_dict_keys trainX
    return full_name.apply(lambda x: generateNoise(x,noise))
# list(map(lambda x:struct_dict_keys.index(x), trainX[0]))
#len(struct_dict_keys)
#full_name_noise = batch_noise(full_name)

def extract_structure(word,n_char=4):
    x_struct = []
    word_len = len(word) + n_char
    n_char-=1
    counter = 0
    for i in range(word_len):
        end = i+1
        start = (i - n_char) if (i - n_char) > 0 else 0
        if word[start:end]!='_':
            x_struct.append(word[start:end])
    return x_struct

#extract_structure('_Elias_Abdalla__Filho_')
#full_name_struct_noise = full_name_noise[trainIndex].apply(lambda x: extract_structure(x))
#data_source_noise = full_name_struct_noise.apply(lambda x: transform_structure(x))

In [18]:
full_name_noise = batch_noise(full_name[trainIndex])
full_name_struct_noise = full_name_noise.apply(lambda x: extract_structure(x))
data_source_noise = full_name_struct_noise.apply(lambda x: transform_structure(x))


In [19]:
data_source_noise

4818     [906, 3355, 1882, 5124, 14855, 0, 20456, 6147,...
20785    [25, 26, 5024, 6238, 6239, 4627, 633, 6629, 41...
43821    [59, 2139, 4970, 585, 0, 37298, 9231, 608, 207...
9438     [250, 251, 231, 232, 233, 234, 507, 16, 13259,...
18000    [14321, 0, 11820, 11821, 6690, 24874, 34910, 3...
20649    [90, 91, 877, 3097, 4947, 7167, 17882, 5814, 0...
13025    [1016, 2488, 5521, 6109, 828, 6110, 6111, 6112...
10672    [1586, 1587, 4405, 6106, 5566, 6107, 6108, 525...
39069    [695, 6240, 6241, 6242, 6243, 12811, 9060, 143...
32225    [740, 741, 1301, 14306, 755, 4160, 655, 690, 5...
35050    [648, 3027, 1946, 9960, 12243, 12244, 12245, 1...
14574    [40, 289, 129, 6366, 6367, 9329, 0, 0, 10197, ...
37496    [59, 2822, 2823, 4144, 1932, 9627, 18900, 1890...
18714    [14321, 0, 21911, 0, 35427, 35428, 13805, 1933...
44007    [14321, 0, 6549, 45384, 10632, 17453, 8735, 23...
32943    [1487, 2777, 2778, 2779, 2780, 2781, 2782, 109...
35933    [1016, 14998, 34875, 0, 0, 2017, 2018, 9118, 6.

In [22]:
data_source_noise = pad_sequences(data_source_noise, maxlen=50,value=0.)

In [23]:
data_source_noise.shape

(35653, 50)

In [24]:
"""
model = Sequential()
model.add(Embedding(len(name_struct_keys)+1,embedding_vector_length,input_length=max_sequence))
model.add(Conv1D(filters=embedding_vector_length,kernel_size=3,padding='same',activation='relu'))
#model.add(MaxPooling1D(pool_size=2))
#model.add(LSTM(lstm_layer,dropout=0.8))
model.add(Bidirectional(LSTM(max_sequence*2,return_sequences=False),input_shape=(max_sequence,1)))
#model.add(TimeDistributed(keras.layers.Dense(len(ethnic_keys),activation='softmax')))
model.add(keras.layers.Dense(len(ethnic_keys),activation='softmax'))
#model.add(keras.layers.Dense(len(ethnic_keys),activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())
"""
for x in range(10):
    full_name_noise = batch_noise(full_name[trainIndex])
    full_name_struct_noise = full_name_noise.apply(lambda x: extract_structure(x))
    data_source_noise = full_name_struct_noise.apply(lambda x: transform_structure(x))
    data_source_noise = pad_sequences(data_source_noise, maxlen=50,value=0.)
    model.fit(data_source_noise,trainY,epochs=1,batch_size=1000)
    scores = model.evaluate(testX,testY,verbose=0)
    print("Accuracy: %.2f%%" %(scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 1000)          62696000  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 50, 1000)          3001000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               880800    
_________________________________________________________________
dense_1 (Dense)              (None, 23)                4623      
Total params: 66,582,423
Trainable params: 66,582,423
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/1
35653/35653 [==============================] - 1360s - loss: 1.9434 - acc: 0.4553   
Accuracy: 64.20%
Epoch 1/1
35653/35653 [==============================] - 1886s - loss: 1.0247 - acc: 0.6984   
Accuracy: 78.82%
Epoch 1/1
35653/35653 [==========

KeyboardInterrupt: 

In [ ]:
"""
for x in range(10):
    full_name_noise = batch_noise(full_name[trainIndex])
    full_name_struct_noise = full_name_noise.apply(lambda x: extract_structure(x))
    data_source_noise = full_name_struct_noise.apply(lambda x: transform_structure(x))
    data_source_noise = pad_sequences(data_source_noise, maxlen=50,value=0.)
    model.fit(data_source_noise,trainY,epochs=1,batch_size=1000)
    scores = model.evaluate(testX,testY,verbose=0)
    print("Accuracy: %.2f%%" %(scores[1]*100))
"""

In [27]:
# load json and create model
json_file = open('model-keras-embed-bilstm-wnoise.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model-keras-embed-bilstm-wnoise-10.h5")
model = loaded_model
print("Loaded model from disk")    

OSError: Unable to open file (Unable to open file: name = 'model-keras-embed-bilstm-wnoise-10.h5', errno = 2, error message = 'no such file or directory', flags = 0, o_flags = 0)

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
for x in range(10):
    full_name_noise = batch_noise(full_name[trainIndex])
    full_name_struct_noise = full_name_noise.apply(lambda x: extract_structure(x))
    data_source_noise = full_name_struct_noise.apply(lambda x: transform_structure(x))
    data_source_noise = pad_sequences(data_source_noise, maxlen=50,value=0.)
    model.fit(data_source_noise,trainY,epochs=1,batch_size=1000)
    scores = model.evaluate(testX,testY,verbose=0)
    print("Accuracy: %.2f%%" %(scores[1]*100))

Epoch 1/1
35653/35653 [==============================] - 1365s - loss: 0.9825 - acc: 0.7210   
Accuracy: 85.10%
Epoch 1/1
35653/35653 [==============================] - 1362s - loss: 0.5206 - acc: 0.8471   
Accuracy: 84.52%
Epoch 1/1
35653/35653 [==============================] - 1373s - loss: 0.3763 - acc: 0.8867   
Accuracy: 83.24%
Epoch 1/1
35653/35653 [==============================] - 1360s - loss: 0.2965 - acc: 0.9111   
Accuracy: 81.32%
Epoch 1/1
35653/35653 [==============================] - 2013s - loss: 0.2502 - acc: 0.9245   
Accuracy: 80.74%
Epoch 1/1
35653/35653 [==============================] - 4596s - loss: 0.1892 - acc: 0.9421   
Accuracy: 77.17%
Epoch 1/1
35653/35653 [==============================] - 4796s - loss: 0.1853 - acc: 0.9445   
Accuracy: 77.47%
Epoch 1/1
35653/35653 [==============================] - 4583s - loss: 0.1671 - acc: 0.9474   
Accuracy: 77.22%
Epoch 1/1
35653/35653 [==============================] - 4597s - loss: 0.1538 - acc: 0.9526   
Accuracy:

In [24]:
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.fit(data_source_noise,trainY,epochs=1,batch_size=1000)
    scores = model.evaluate(testX,testY,verbose=0)
    print("Accuracy: %.2f%%" %(scores[1]*100))

Epoch 1/1
35653/35653 [==============================] - 1369s - loss: 0.9920 - acc: 0.7179   
Accuracy: 85.19%


In [ ]:
print('aha')